In [1]:
import pandas as pd
import numpy as np

import tqdm
from geopy.distance import geodesic
import random
import datetime
# from datetime import datetime, timedelta, strftime  # Импортируйте strftime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# adresses = pd.read_csv('london_postcodes-ons-postcodes-directory-feb22.csv', delimiter=',')
adresses = pd.read_csv('../Lab3_MongoDB/London_postcodes.csv', delimiter=',')


In [3]:
taxi_rides = pd.DataFrame(
    columns=['driver_id', 'trip_id', 'client_id', 'start_point', 'end_point', 'distance', 'start_time',
             'end_time', 'road_time', 'cost', 'driver_rate', 'client_rate', 'category_driver_feedback', 
             'text_driver_feedback', 'category_client_feedback', 'text_client_feedback'
             ]
    )

In [4]:
num_rows = 20_000_000
num_drivers = 5000
num_clients = 2000

start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2024, 1, 1)


driver_id_range = range(1, num_drivers + 1)
client_id_range = range(1, num_clients + 1)


random_addresses = adresses.sample(n=num_rows, random_state=42, replace=True)

# Заполняем столбцы start_point и end_point случайными парами широты и долготы
taxi_rides['start_point'] = list(zip(random_addresses['Latitude'], random_addresses['Longitude']))
taxi_rides['end_point'] = list(zip(random_addresses['Latitude'].sample(frac=1, random_state=42), random_addresses['Longitude'].sample(frac=1, random_state=42)))

In [5]:
taxi_rides['driver_id'] = [random.choice(driver_id_range) for i in range(taxi_rides.shape[0])]
taxi_rides['client_id'] = [random.choice(client_id_range) for i in range(taxi_rides.shape[0])]

In [6]:
# Рассчитываем расстояние
taxi_rides['distance'] = [geodesic(start, end).km for start, end in taxi_rides[['start_point', 'end_point']].values]
taxi_rides['distance'] = taxi_rides['distance'].round(2)


In [7]:

taxi_rides['start_time'] = [datetime.datetime.strftime(start_date + datetime.timedelta(seconds=random.randint(0, 31536000)), '%Y-%m-%d %H:%M:%S') for i in range(num_rows)]
taxi_rides['road_time'] = [datetime.timedelta(seconds=distance / 50 * 3600) for distance in taxi_rides['distance']]
taxi_rides['end_time'] = [datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + road_time for start_time, road_time in zip(taxi_rides['start_time'], taxi_rides['road_time'])]

taxi_rides['start_time'] = pd.to_datetime(taxi_rides['start_time'], format='%Y-%m-%d %H:%M:%S')
taxi_rides['end_time'] = pd.to_datetime(taxi_rides['end_time'], format='%Y-%m-%d %H:%M:%S')

taxi_rides['trip_id'] = range(1, len(taxi_rides) + 1)


In [8]:
def count_cost(start_time, distance):
    cost = 5 + .5 * distance
    if (start_time.hour >= 8 and start_time.hour <= 11) or (start_time.hour >= 17 and start_time.hour <= 19):
        cost *= 1.5

    if (start_time.hour >= 23 and start_time.hour <= 6):
        cost *= 1.75
    return cost


taxi_rides['cost'] = [count_cost(time, dist) for time, dist in zip(taxi_rides['start_time'], taxi_rides['distance'])]
taxi_rides['cost'] = taxi_rides['cost'].round(2)

In [9]:
driver_rate_idx = np.random.randint(low=0, high=num_drivers, size=int(num_rows*0.85))
driver_rate_distr_arr = np.random.multinomial(1, [.2, .05, .1, .25, .4], size=int(num_rows*0.85))
taxi_rides['driver_rate'][driver_rate_idx] = np.where(driver_rate_distr_arr == 1)[1] + 1

client_rate_idx = np.random.randint(low=0, high=num_drivers, size=int(num_rows*0.75))
client_rate_distr_arr = np.random.multinomial(1, [.2, .05, .1, .25, .4], size=int(num_rows*0.75))
taxi_rides['client_rate'][driver_rate_idx] = np.where(driver_rate_distr_arr == 1)[1] + 1

In [10]:
driver_fb_cat_good = ['good service', 'nice car', 'expert navigation']
driver_fb_cat_bad = ['awful service', 'bad car', 'dirty', 'rude']

client_fb_cat_good = ['polite', 'pleasant', 'quiet']
client_fb_cat_bad = ['unpolite', 'unpleasant', 'loud', 'dirty']

category_driver_good_feedback_idx = np.random.choice(taxi_rides[taxi_rides.driver_rate > 3].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_driver_feedback"][category_driver_good_feedback_idx] = np.random.choice(driver_fb_cat_good, size=int(num_rows*0.3*0.2))

category_driver_bad_feedback_idx = np.random.choice(taxi_rides[taxi_rides.driver_rate < 4].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_driver_feedback"][category_driver_bad_feedback_idx] = np.random.choice(driver_fb_cat_bad, size=int(num_rows*0.3*0.2))


text_good_feedback_driver_length = np.random.randint(low=0, high=len(driver_fb_cat_good), size=int(num_rows*0.3*0.2))
text_good_feedback_driver_sample = [random.sample(driver_fb_cat_good, i) for i in text_good_feedback_driver_length]
taxi_rides['text_driver_feedback'][category_driver_good_feedback_idx] = text_good_feedback_driver_sample

text_bad_feedback_driver_length = np.random.randint(low=0, high=len(driver_fb_cat_bad), size=int(num_rows*0.3*0.2))
text_bad_feedback_driver_sample = [random.sample(driver_fb_cat_bad, i) for i in text_bad_feedback_driver_length]
taxi_rides['text_driver_feedback'][category_driver_bad_feedback_idx] = text_bad_feedback_driver_sample

# ------------------

category_client_good_feedback_idx = np.random.choice(taxi_rides[taxi_rides.client_rate > 3].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_client_feedback"][category_client_good_feedback_idx] = np.random.choice(client_fb_cat_good, size=int(num_rows*0.3*0.2))

category_client_bad_feedback_idx = np.random.choice(taxi_rides[taxi_rides.client_rate < 4].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_client_feedback"][category_client_bad_feedback_idx] = np.random.choice(client_fb_cat_bad, size=int(num_rows*0.3*0.2))


text_good_feedback_client_length = np.random.randint(low=0, high=len(client_fb_cat_good), size=int(num_rows*0.3*0.2))
text_good_feedback_client_sample = [random.sample(client_fb_cat_good, i) for i in text_good_feedback_client_length]
taxi_rides['text_client_feedback'][category_client_good_feedback_idx] = text_good_feedback_client_sample

text_bad_feedback_client_length = np.random.randint(low=0, high=len(client_fb_cat_bad), size=int(num_rows*0.3*0.2))
text_bad_feedback_client_sample = [random.sample(client_fb_cat_bad, i) for i in text_bad_feedback_client_length]
taxi_rides['text_client_feedback'][category_client_bad_feedback_idx] = text_bad_feedback_client_sample


In [11]:
taxi_rides

,driver_id,trip_id,client_id,start_point,end_point,distance,start_time,end_time,road_time,cost,driver_rate,client_rate,category_driver_feedback,text_driver_feedback,category_client_feedback,text_client_feedback
0,4570,1,85,"(51.62878, -0.172348)","(51.391488, -0.054597)",27.64,2023-01-05 22:10:35,2023-01-05 22:43:45.080,0 days 00:33:10.080000,18.82,5,5,expert navigation,"[expert navigation, nice car]",quiet,[quiet]
1,3221,2,1919,"(51.551131, -0.173485)","(51.446049, -0.025152)",15.58,2023-01-09 16:00:39,2023-01-09 16:19:20.760,0 days 00:18:41.760000,12.79,5,5,nice car,[],pleasant,[quiet]
2,3408,3,682,"(51.580247, -0.1188)","(51.409559, -0.30682)",23.05,2023-06-21 14:21:59,2023-06-21 14:49:38.600,0 days 00:27:39.600000,16.52,1,1,awful service,[],loud,"[unpleasant, loud]"
3,4376,4,866,"(51.546822, -0.442101)","(51.524565, -0.112042)",23.04,2023-04-04 21:07:37,2023-04-04 21:35:15.880,0 days 00:27:38.880000,16.52,5,5,nice car,[good service],pleasant,"[pleasant, quiet]"
4,3072,5,1153,"(51.566182, -0.137344)","(51.43585, 0.101123)",22.01,2023-05-03 15:05:40,2023-05-03 15:32:04.720,0 days 00:26:24.720000,16.01,1,1,awful service,"[dirty, bad car]",unpolite,"[loud, unpolite, unpleasant]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,360,19999996,1058,"(51.60295, -0.26264)","(51.585036, -0.202872)",4.60,2023-11-14 22:36:28,2023-11-14 22:41:59.200,0 days 00:05:31.200000,7.30,NaN,NaN,NaN,NaN,NaN,NaN
19999996,1196,19999997,11,"(51.515084, -0.140539)","(51.519135, -0.434643)",20.42,2023-07-18 14:36:17,2023-07-18 15:00:47.240,0 days 00:24:30.240000,15.21,NaN,NaN,NaN,NaN,NaN,NaN
19999997,143,19999998,978,"(51.535679, -0.481051)","(51.425729, -0.218363)",21.97,2023-04-25 21:44:27,2023-04-25 22:10:48.840,0 days 00:26:21.840000,15.98,NaN,NaN,NaN,NaN,NaN,NaN
19999998,1486,19999999,1982,"(51.361836, -0.096563)","(51.421414, 0.069407)",13.32,2023-08-07 18:30:43,2023-08-07 18:46:42.040,0 days 00:15:59.040000,17.49,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
taxi_rides.to_csv('taxi.csv',  index=False)

In [12]:
taxi_rides.to_csv('data.txt', sep=',', index=False)